<a href="https://colab.research.google.com/github/muxeres/Analise-Fraude-Bancario/blob/master/C%C3%B3pia_de_%5BONLINE%5D_Exemplo_2_Agente_de_di%C3%A1logo_baseado_em_regras_e_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exemplo 2 - Agente de diálogo híbrido (baseado em regras e dados)
## Tecnólogo em Inteligência Artificial Aplicada - Agentes Conversacionais
Neste notebook iremos construir um agente de diálogo que trará ocorrências sobre determinado tema.

### Qual o contexto de nosso agente?

Iremos desenvolver um agente de diálogo de question answering, que baseado em um corpus de texto sobre um assunto, trará as informações mais relevantes de acordo com a consulta do usuário.

### Quais ferramentas e técnicas iremos utilizar?

*   **NLTK** - O mais famoso toolkit de Processamento de Linguagem Natural em Python.
*   **Expressões Regulares** - o pacote de regex do Python será utilizado para otimizar a busca de padrões.
*   **urllib e BeautifulSoup** - Bibliotecas para obter dados de páginas HTML.
*   **scikit-learn** - Pacote com funcionalidades de manipulaçã de dados e Machine Learning, vamos utilizar TF-IDF e Similaridade de cosseno.



### Construindo o agente de diálogo
Nosso agente vai operar da seguinte maneira:

1.   Recebe **entrada** do usuário
2.   **Pré-processa** a entrada do usuário
3.   Calcula a **similaridade** entre a entrada e as sentenças do corpus
4.   Obtém a sentença **mais similar do corpus**
5.   Mostra-a como **resposta** ao usuário

Anteriormente a estas etapas, iremos criar nosso corpus ao obter dados da Wikipedia, automaticamente, então vamos lá!


#### Importando as bibliotecas
Vamos importar o pacote de expressões regulares do Python e também o acesso ao WordNet dado pelo NLTK.

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('rslp')# Stemming em pt-br
from nltk.corpus import stopwords
nltk.download('stopwords')# Lista de stopwords
import numpy as np
import random
import string
import bs4 as bs
import urllib.request
import re
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#### Construindo o corpus
Vamos fazer um *web-scraping* para obter os dados automaticamente da wikipedia. Este processo precisa ser executado apenas uma vez, e o arquivo salvo em forma de texto na máquina.

In [ ]:
# Aqui buscamos a página sobre o BRASIL. Caso queira mudar o tema basta colocar o link para outra página.
# Você pode também optar por obter dados de várias páginas diferentes, basta definir uma lista de links e iterar sobre elas.
codigo_html = urllib.request.urlopen('https://pt.wikipedia.org/wiki/Brasil')
codigo_html = codigo_html.read()

# Processa o código HTML lido
html_processado = bs.BeautifulSoup(codigo_html, 'lxml')

# Busca todos parágrafos do texto
paragrafos = html_processado.find_all('p')

texto = ''

# Percorre parágrafos e concatena textos
for p in paragrafos:
  texto += p.text

# Normaliza para minúsculas
texto = texto.lower()
texto[0:1000]

##### Pré-processando o corpus
Como você pode ver no trecho acima, precisamos remover caracteres especiais do texto, além de dividí-lo em sentenças válidas.

In [ ]:
texto = re.sub(r'\[[0-9]*\]', ' ', texto)
texto = re.sub(r'\s+', ' ', texto)
texto[0:1000]

In [ ]:
sentencas = nltk.sent_tokenize(texto, language='portuguese')
palavras = nltk.word_tokenize(texto, language='portuguese')
sentencas[10:15]

#### Funções de pré-processamento de entrada do usuário
Vamos criar funções para pré-processar as entradas do usuário, vamos retirar pontuações e usar Stemming nos textos, para que palavras similares sejam processadas de maneira igual pelo algoritmo (e.g., pedra e pedregulho teriam a mesma forma léxica)

In [ ]:
# Define uma função que faz Stemming em todo um texto
def stemming(tokens):
  stemmer = nltk.stem.RSLPStemmer()
  novotexto = []
  for token in tokens:
    novotexto.append(stemmer.stem(token.lower()))
  return novotexto

# Função que remove pontuação
removePontuacao = dict((ord(punctuation), None) for punctuation in string.punctuation)

def preprocessa(documento):
  return stemming(nltk.word_tokenize(documento.lower().translate(removePontuacao), language='portuguese'))

In [ ]:
# Veja como fica um texto após o pré-processamento
preprocessa("Olá meu nome é Lucas, eu moro no Brasil, e você?")

#### Resposta à saudações
Mesmo que estejamos construindo um sistema de diálogo (baseado em tarefas), é normal que o usuário inicie conversas com saudações ao agente. Portanto, iremos desenvolver rapidamente uma função (i.e., regras) para lidar especialmente com esta situação.
Vamos criar algumas respostas possíveis, e sempre vamos escolher aleatóriamente uma delas, para evitar que nosso agente fique repetitivo.

In [ ]:
saudacoes_entrada = ("olá", "bom dia", "boa tarde", "boa noite", "oi", "como vai", "e aí")
saudacoes_respostas = ["olá", "olá, espero que esteja tudo bem contigo", "oi", "Oie", "Seja bem-vindo, em que posso te ajudar?"]

def geradorsaudacoes(saudacao):
  for token in saudacao.split():
    if token.lower() in saudacoes_entrada:
      return random.choice(saudacoes_respostas)

In [ ]:
# Execute este exemplo várias vezes que verá diferentes respostas
geradorsaudacoes('Olá')

#### Resposta à consultas do usuário
Agora, teremos uma função para lidar com consultas do usuário. Onde iremos comparar a similaridade entre a entrada do usuário, com as sentenças do corpus. Caso encontremos, a sentença mais similar será mostrada como resposta.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def geradorrespostas(entradausuario):
  resposta = ''
  sentencas.append(entradausuario)

  word_vectorizer = TfidfVectorizer(tokenizer=preprocessa, stop_words=stopwords.words('portuguese'))
  all_word_vectors = word_vectorizer.fit_transform(sentencas)
  similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)
  similar_sentence_number = similar_vector_values.argsort()[0][-2]

  matched_vector = similar_vector_values.flatten()
  matched_vector.sort()
  vector_matched = matched_vector[-2]

  if vector_matched == 0:
    resposta = resposta + "Me desculpe, não entendi o que você pediu."
    return resposta
  else:
    resposta = resposta + sentencas[similar_sentence_number]
    return resposta

#### Interagindo com o agente de diálogo
Vamos definir um algoritmo que continue interagindo com o usuário até que ele decida finalizar.

O resultado não é sempre o ideal, mas já cobre muitas possíveis perguntas. Se utilizássemos apenas regras de diálogo para responder perguntas sobre um tema, precisaríamos de centenas de regras para tal. Mas como baseamos nossas respostas em dados apenas uma regra que calcula similaridade com nosso corpus já é o suficiente.

Faça perguntas como:
*  *Qual o esporte mais popular no Brasil?*
*  *Quais eventos esportivos o Brasil já organizou?*
*  *Como é a cozinha brasileira?*
*  *Onde são realizadas pesquisas tecnológicas no Brasil?*


In [ ]:
continue_dialogue = True
print("Olá, eu sou o Agente Tupiniquim. Me pergunte qualquer coisa sobre nosso país.")
while (continue_dialogue == True):
  # Obtém entrada do usuário
  human_text = input().lower()

  if human_text != 'tchau':
    if human_text == 'obrigado' or human_text == 'muito obrigado' or human_text == 'agradecido':
      continue_dialogue = False
      print("Agente Tupiniquim: Disponha")
    else:
      if geradorsaudacoes(human_text) != None:
        print("Agente Tupiniquim: " + geradorsaudacoes(human_text))
      else:
        print("Agente Tupiniquim: ", end="")
        print(geradorrespostas(human_text))
        sentencas.remove(human_text)
  else:
    continue_dialogue = False
    print("Agente Tupiniquim: Até a próxima.")

### O que pode ser feito?
Utilizamos um modelo baseado em regras, onde uma das regras faz uso de um corpus de dados para formular as respostas, o que já deixou nosso modelo bem mais flexível, sem necessidade de criação de centenas/milhares de regras.

Mas, o que poderíamos fazer para melhorar?


*   Poderíamos obter não apenas os parágrafos(`<p>`) na página da wikipedia, mas também utilizar os dados dispostos na coluna direita, que apresentam informações bem relevantes como população, atual presidente, etc., para montar sentenças.
*   Poderíamos melhorar ainda mais o cálculo de similaridade ao utilizar um modelo de Word Embeddings, além do TF-IDF.
*   Obter dados sobre o Brasil de diferentes fontes.
*   Criar um classificador de contexto para o agente, e de maneira dinâmica buscar páginas da Wikipedia correspondentes à pergunta do usuário, e só então dar a resposta. Assim nosso agente não ficaria limitado a perguntas sobre o Brasil.



## Referências e Material complementar

* [Python for NLP: Creating a Rule-Based Chatbot](https://stackabuse.com/python-for-nlp-creating-a-rule-based-chatbot/)
* [Building a Simple Chatbot from Scratch in Python (using NLTK)](https://morioh.com/p/6cc33336784c)
* [Building a simple chatbot in python](https://medium.com/nxtplus/building-a-simple-chatbot-in-python-3963618c490a)
* [Designing A ChatBot Using Python: A Modified Approach](https://towardsdatascience.com/designing-a-chatbot-using-python-a-modified-approach-96f09fd89c6d)
* [Build Your First Python Chatbot Project](https://dzone.com/articles/python-chatbot-project-build-your-first-python-pro)
* [Python Chatbot Project – Learn to build your first chatbot using NLTK & Keras](https://data-flair.training/blogs/python-chatbot-project/)
* [Python Chat Bot Tutorial - Chatbot with Deep Learning (Part 1)](https://www.youtube.com/watch?v=wypVcNIH6D4)
* [Intelligent AI Chatbot in Python](https://www.youtube.com/watch?v=1lwddP0KUEg)
* [Coding a Jarvis AI Using Python 3 For Beginners](https://www.youtube.com/watch?v=NZMTWBpLUa4)

Este notebook foi produzido por Prof. [Lucas Oliveira](http://lattes.cnpq.br/3611246009892500).